<a href="https://colab.research.google.com/github/szucshey/osm-data-exploration/blob/main/census/census_data_in_hungary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install folium osm2geojson

In [2]:
import folium
import requests
import json
import pandas as pd
from shapely import geometry
import random
import osm2geojson

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [3]:
#Magyarország településhatárainak, megyehatárainak, a főváros kerületeinek határainak lekérése az OSM API-n keresztül

#Elsőként a településhatárok (admin_level=8), kivéve:
#   -Balatonakarattyát, mert 2014-ben vált csak önálló településsé
#   -Budapestet, mert a területét a kerületek (admin_level=9)  már egyszer lefedik
overpass_url = "http://overpass-api.de/api/interpreter"
overpass_query = """
    [out:json];
    rel["wikipedia"~"^hu:"]
    [admin_level=8]
    [type=boundary]
    [name!~"Balatonakarattya"]
    [name!="Budapest"]
    [boundary=administrative];

    out geom;
    rel["wikipedia"~"^hu:"]
    [admin_level=9]
    [type=boundary]
    [boundary=administrative];
    out geom;
    """

response = requests.get(overpass_url, params={'data': overpass_query})

#Az API válasz lementése egy json fileba
output = open("towns.json", "w")
json.dump(response.json(), output)
output.close()

######################################################################################################################
#Végül a megyehatárok (admin_level=6) + az országhatár (admin_level=2), a megjelenítés stílusának kialakítása miatt kérjük le ezeket is
overpass_query = """
    [out:json];
    rel["ISO3166-2"~"^HU"]
    [admin_level=6]
    [type=boundary]
    [boundary=administrative];
    out geom;
    
    rel["ISO3166-1"~"^HU"]
    [admin_level=2]
    [type=boundary]
    [boundary=administrative];
    out geom;
    """

response = requests.get(overpass_url, params={'data': overpass_query})

#Az API válasz lementése egy json fileba
output = open("counties.json", "w")
json.dump(response.json(), output)
output.close()

In [4]:
df = pd.read_csv("data.csv", delimiter=";")
#A népszámlálások lehetséges évszámai, az egyes években eltérő módon kategorizálták a nemzetiségieket, ezért van az évszámokhoz egy-egy kategória is rendelve
#   https://mtatkki.ogyk.hu/nepszamlalas_leiras.php

ev_kategoria = [[1880, "A"], [1890, "A"], [1900, "A"], [1910, "A"], [1920, "A"], [1930, "A"], [1941, "N"], [1949, "N"], [1960, "N"], [1980, "N"], [1990, "N"], [2001, "A-N-H-K"], [2011, "A-N-H-K"]]
nemzetisegek, fgs, styles_county_borders, styles_town_borders = [], [], [], []
color_codes = ["#FFF712", "#09EC8E", "#D57C1E", "#1C24BA", "#DC1111", "#0BF7FF", "#FF74E8", "#A2059D", "#61BD00"]

############################################
#A tömb azon eleme, amelyik év adatait szeretnénk megjeleníteni
year = 11
############################################

df = df[(df["Év"] == ev_kategoria[year][0]) & (df["Kategória"] == ev_kategoria[year][1]) & (df["Mai terület"] == "#") & (df["Település"] != "Budapest")]

colors_added = 0
#Az egyes nemzetiségek számbavétele is népszámlálásonként eltért, úgyhogy mindig csak az adott évben számlált nemzetiségeket vesszük figyelembe
for i in range(9, len(df.columns)-4):
    if df[df.columns[i]].values.sum() > 0:
        nemzetisegek.append(df.columns[i])
        fgs.append(folium.FeatureGroup(name="<span style=\"font-size: 20px; color: {0}; text-shadow: -1px 0 black, 0 1px black, 1px 0 black, 0 -1px black;\">{1}</span>".format(color_codes[colors_added], df.columns[i]), overlay=False, control=True, show=False))
        colors_added += 1
        
styles_county_borders = [lambda x: {'fillColor': '#000000', 'fillOpacity': 0.0, 'stroke': True, 'color': "#FFF712", 'weight': 1.5},
         lambda x: {'fillColor': '#000000', 'fillOpacity': 0.0, 'stroke': True, 'color': "#09EC8E", 'weight': 1.5},
         lambda x: {'fillColor': '#000000', 'fillOpacity': 0.0, 'stroke': True, 'color': "#D57C1E", 'weight': 1.5},
         lambda x: {'fillColor': '#000000', 'fillOpacity': 0.0, 'stroke': True, 'color': "#1C24BA", 'weight': 1.5},
         lambda x: {'fillColor': '#000000', 'fillOpacity': 0.0, 'stroke': True, 'color': "#DC1111", 'weight': 1.5},
         lambda x: {'fillColor': '#000000', 'fillOpacity': 0.0, 'stroke': True, 'color': "#0BF7FF", 'weight': 1.5},
         lambda x: {'fillColor': '#000000', 'fillOpacity': 0.0, 'stroke': True, 'color': "#FF74E8", 'weight': 1.5},
         lambda x: {'fillColor': '#000000', 'fillOpacity': 0.0, 'stroke': True, 'color': "#A2059D", 'weight': 1.5},
         lambda x: {'fillColor': '#000000', 'fillOpacity': 0.0, 'stroke': True, 'color': "#61BD00", 'weight': 1.5}]

styles_town_borders = [lambda x: {'fillColor': '#000000', 'fillOpacity': 0.0, 'stroke': True, 'color': "#FFF712", 'weight': 0.5},
         lambda x: {'fillColor': '#000000', 'fillOpacity': 0.0, 'stroke': True, 'color': "#09EC8E", 'weight': 0.5},
         lambda x: {'fillColor': '#000000', 'fillOpacity': 0.0, 'stroke': True, 'color': "#D57C1E", 'weight': 0.5},
         lambda x: {'fillColor': '#000000', 'fillOpacity': 0.0, 'stroke': True, 'color': "#1C24BA", 'weight': 0.5},
         lambda x: {'fillColor': '#000000', 'fillOpacity': 0.0, 'stroke': True, 'color': "#DC1111", 'weight': 0.5},
         lambda x: {'fillColor': '#000000', 'fillOpacity': 0.0, 'stroke': True, 'color': "#0BF7FF", 'weight': 0.5},
         lambda x: {'fillColor': '#000000', 'fillOpacity': 0.0, 'stroke': True, 'color': "#FF74E8", 'weight': 0.5},
         lambda x: {'fillColor': '#000000', 'fillOpacity': 0.0, 'stroke': True, 'color': "#A2059D", 'weight': 0.5},
         lambda x: {'fillColor': '#000000', 'fillOpacity': 0.0, 'stroke': True, 'color': "#61BD00", 'weight': 0.5}]


In [5]:
#A térkép létrehozása foliumban, kezdeti koordináták, zoom-lehetőségek, térképvászon megadása, az attribútum kötelező külső vászon használata esetén
m = folium.Map(location=[47.161, 19.505], zoom_start=8, min_zoom=7, maxBounds=[[40.000, 10.000], [60.000, 30.000]], tiles=None, attr="", prefer_canvas=True, overlay=True)
#https://{s}.basemaps.cartocdn.com/dark_nolabels/{z}/{x}/{y}{r}.


style_function = lambda x: {'fillColor': '#000000', 'fillOpacity': 0.0, 'opacity': 0.2, 'stroke': True, 'color': "#fff", 'weight': 1}
data = json.load(open("counties.json", "r"))
geojson_counties = osm2geojson.json2geojson(data)

style_function = lambda x: {'fillColor': '#000000', 'fillOpacity': 0.0, 'opacity': 0.2, 'stroke': True, 'color': "#fff", 'weight': 0.3}
data = json.load(open("towns.json", "r"))
geojson_towns = osm2geojson.json2geojson(data)


/home/szucshey/.local/lib/python3.8/site-packages/osm2geojson/main.py:514: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for line in merged_line:


In [6]:
#Alakzaton belüli pont generálás
def get_random_point_in_polygon(poly):
  
  minx, miny, maxx, maxy = poly.bounds
  while True:
       p = geometry.Point(random.uniform(minx, maxx), random.uniform(miny, maxy))
       if poly.contains(p):
          return p

In [7]:
layers = folium.LayerControl(position='bottomleft')

#adott megyén belül minden x (representation) főre generálódik egy pont
representation = 1

for n in range(len(nemzetisegek)):
    
    #nemzetiségenként a megfelelő paraméterek beállítása
    nemzetiseg = nemzetisegek[n]  
    df2 = df[["Település", nemzetiseg]]
    fill = color_codes[n]
    fgs[n].add_child(folium.GeoJson(geojson_counties, name='counties_colored', style_function=styles_county_borders[n], smooth_factor=2.0, control=False))
    fgs[n].add_child(folium.GeoJson(geojson_towns, name='towns_colored', style_function=styles_town_borders[n], smooth_factor=2.0, control=False)) 
    
    #minden településen belül annyi pontot generálunk, ahányan az adott nemzetiséghez tartozónak vallották magukat
    for i in range(len(geojson_towns['features'])):
        poly = geometry.Polygon([[coord[0], coord[1]] for coord in geojson_towns['features'][i]['geometry']['coordinates'][0][0]]) 
        
        try:
            if geojson_towns['features'][i]['properties']['tags']['name'] in df2['Település'].values:
                geojson_towns['features'][i]['properties']['popup{0}'.format(n)] = "<b>{0}</b>".format(geojson_towns['features'][i]['properties']['tags']['name']) + ": {0}".format(int(df2[df2["Település"] == geojson_towns['features'][i]['properties']['tags']['name']][nemzetiseg]))
                list(fgs[n]._children.values())[1].add_child(folium.features.GeoJsonTooltip(['popup{0}'.format(n)], labels=False, sticky=False))
                
                for j in range(int(df2[df2["Település"] == geojson_towns['features'][i]['properties']['tags']['name']][nemzetiseg]) // representation):
                    point = get_random_point_in_polygon(poly)
                    fgs[n].add_child(folium.Circle(location=[point.y, point.x], radius=5, fill_color=fill, fill_opacity=1.0, opacity=0.0, interactive=False))
                
            else:
                geojson_towns['features'][i]['properties']['popup{0}'.format(n)] = "<b>{0}:</b> nincs adat".format(geojson_towns['features'][i]['properties']['tags']['name'])
        except:
            print(geojson_towns['features'][i]['properties']['tags']['name'],
                df2[df2["Település"] == geojson_towns['features'][i]['properties']['tags']['name']][nemzetiseg]) 
    
    m.add_child(fgs[n])

m.add_child(layers)

css_modify = "<style>.leaflet-container {background: #080808}</style>"
m.get_root().html.add_child(folium.Element(css_modify))

m.save(outfile="map.html")